# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [29]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [30]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 最後的情書, ID: 10880
Movie: 聖雅各的天空, ID: 10877
Movie: 看不見的證人, ID: 10873
Movie: 午夜0時的吻, ID: 10855
Movie: 爆裂魔神少女, ID: 10853
Movie: 初心, ID: 10837
Movie: 高盧英雄歷險記：魔法藥水, ID: 10834
Movie: 下一站，托斯卡尼, ID: 10817
Movie: 冬天的故事 經典數位修復, ID: 10739
Movie: 秋天的故事 經典數位修復, ID: 10738
Movie: 可不可以，你也剛好喜歡我, ID: 10473
Movie: 羊與鋼之森, ID: 8460
Movie: 棕櫚泉不思議, ID: 10864
Movie: 東京教父：4K數位修復版, ID: 10860
Movie: Fate/stay night [Heaven's Feel] III.春櫻之歌, ID: 10630
Movie: 椿花散落, ID: 10852
Movie: 劇場版 新幹線變形機器人—來自未來的神速ALFA-X, ID: 10823
Movie: 我的兒子是死刑犯, ID: 10782
Movie: 夏天的故事 經典數位修復, ID: 10737
Movie: 春天的故事 經典數位修復, ID: 10736
Movie: 逃出立法院, ID: 10535
Movie: 全面啟動10周年紀念版, ID: 10867
Movie: 天劫倒數, ID: 10850
Movie: 魔鬼對決, ID: 10842
Movie: 皮行者, ID: 10794
Movie: 逆者, ID: 10875
Movie: 巴亞拉魔幻冒險, ID: 10851
Movie: 隱藏的畫家, ID: 10841
Movie: 地下社會, ID: 10840
Movie: 心靈咖啡館的驅魔師, ID: 10830
Movie: 6號鬼病床, ID: 10826
Movie: 家靈, ID: 10798
Movie: 陰眼, ID: 10795
Movie: 哈囉少女, ID: 10780
Movie: 怪胎, ID: 10733
Movie: 原本以為只是手機掉了2：被囚禁的殺人魔, ID: 10713
Movie: 記憶拼圖：20週年數

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [120]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10630

In [121]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 花蓮, 代號(area_id): 12


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [122]:
# 指定放映地區
area_id = 28

In [123]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

八月 22
八月 23
八月 24
八月 25
八月 26


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [124]:
# 選定要觀看的日期
date = "2020-08-22"

In [125]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [127]:
#------------------------------name
name=[]
for a in html:
    name.append(a.find('a').text)   

#------------------------------type
movie_type=[]
for b in html:
    movie_type.append(b.find_all("span", attrs={"class":"tapR"}))


#-----------------------------time    
time=[]    
for c in html:
    time.append(c.find_all("input", attrs={"class":"gabtn"}))
#----------------------------time




#------------------------------cinema
cinema=[]
for x in range(len(name)):
    cinema.append(time[x][0]['data-cinema'])
#------------------------------cinema



movie_time=[]
for x in range(len(name)):
    for y in range(len(time[x])):
        movie_time.append(time[x][y]['data-cinema']+' '+time[x][y]['data-movie_type']+time[x][y]['data-movie_date']+' '+time[x][y]['data-movie_time'])

#------------------------------------------------------------cinema_number        
cinema_number=[] 
for x in range(len(name)):
    for y in range(len(time[x])):
        cinema_number.append(time[x][y]['data-cinema'])
#------------------------------------------------------------dict=d    
dict_cinema=dict(zip(cinema,name))       

d=[]
for i in range(len(cinema_number)):
    d.append(dict_cinema[cinema_number[i]])

#-------------------------------------------------------------cinema_date
cinema_date=[] 
for x in range(len(name)):
    for y in range(len(time[x])):
        cinema_date.append(time[x][y]['data-movie_date'])
#-------------------------------------------------------------cinema_time
cinema_time=[] 
for x in range(len(name)):
    for y in range(len(time[x])):
        cinema_time.append(time[x][y]['data-movie_time'])
#-------------------------------------------------------------cinema_type
cinema_type=[] 
for x in range(len(name)):
    for y in range(len(time[x])):
        cinema_type.append(time[x][y]['data-movie_type'])
#-------------------------------------------------------------result
for i in range(len(cinema_number)):
    print(d[i]+'---'+cinema_type[i]+'---'+cinema_date[i]+'--'+cinema_time[i])

    


台北日新威秀影城---數位---08.22--14:25
台北日新威秀影城---數位---08.22--18:15
台北日新威秀影城---數位---08.22--20:50
台北信義威秀影城---數位---08.22--14:40
台北信義威秀影城---數位---08.22--17:00
台北信義威秀影城---數位---08.22--19:20
台北信義威秀影城---數位---08.22--21:40
台北信義威秀影城---數位---08.23--00:00
京站威秀影城---數位---08.22--14:20
京站威秀影城---數位---08.22--16:40
京站威秀影城---數位---08.22--19:00
京站威秀影城---數位---08.22--21:20
MUVIE CINEMAS台北松仁威秀---TITAN---08.22--16:40
MUVIE CINEMAS台北松仁威秀---TITAN---08.22--19:00
